In [ ]:
import random
import numpy as np
import heapq
from tqdm import tqdm
import copy
import pandas as pd

In [ ]:
class player():
    """Just to store information"""
    def __init__(self, chip = 11 ):
        self.chip = chip
        self.card = []
    
    def __repr__(self):
        return f'Chip: {self.chip} | Card owned: {self.card}'
    
    def action(self, move):
        if move == 'take':
            self.card.append(self.game.current_card)
            self.chip += self.game.chip_in_pot
        elif move == 'pass':
            self.chip -= 1
            self.game.chip_in_pot += 1
    
    def calculate_score(self):
        sorted_card = sorted(self.card)
        total_score = 0
        if len(sorted_card) == 0:
            return self.chip
        total_score -= sorted_card[0]
        for score, prev_score in zip(sorted_card[1:], sorted_card[:-1]):
            if score - prev_score > 1:
                total_score -= score
        total_score += self.chip
        return total_score

In [ ]:
class game:
    full_deck = [i for i in range(3,36)]
    
    def __init__(self,
                 card: int = None,
                 n_player: int = 3,
                 n_chip: int = 11,
                 n_remove_card: int = 9
                 ):
        self.n_remove_card = n_remove_card
        self.remain_card = game.full_deck
        self.played_card = []
        self.current_card = self.flip_card(card) if card else self.flip_card()
        
        self.chip_in_pot = 0
        self.turn = 0 # need this to know which player to take
        self.n_player = n_player
        self.n_chip = n_chip

        self.max_score = 1
        self.min_score = -0.1
        self.score_range = self.max_score - self.min_score
        self.rollout_rule = {'pass': 0.9,
                             'take': 0.1}
        self.is_continue = True
        self.init_player(n_player)
    
    def __len__(self):
        return len(self.remain_card)
    
    def __str__(self):
        return f'Current card: {self.current_card} | Played cards: {self.played_card}'
    def __repr__(self):
        return f'Current card: {self.current_card} | Played cards: {self.played_card}'

    def init_player(self, n_player):
        self.players = [player(chip =  self.n_chip) for _ in range(n_player)]
        
    def set_remain_card(self):
        """Update list of remaining cards"""
        self.remain_card = [i for i in game.full_deck 
                            if i not in self.played_card]

    def set_removed_card(self):
        "Remove 9 cards"
        self.removed_card = self.get_cards(9)
        self.set_remain_card()

    def get_cards(self, n):
        'Randomize a card, if number of card remained == 9'
        if len(self.remain_card) == self.n_remove_card:
            return False
        card = random.sample(self.remain_card, n)
        return card[0] if n == 1 else card
    
    
    def flip_card(self, 
                card: int = None):
        '''Progress the game by flipping a new card'''
        if card == None:
            card = self.get_cards(1)
            if not card:
                return card
        self.played_card.append(card)
        self.set_remain_card()
        self.current_card = card
        return card
    
    def calculate_ranking(self):
        score_list = np.array([player.calculate_score() for player in self.players])
        rank_tmp = score_list.argsort()
        ranking = rank_tmp.argsort()
        step = self.score_range/(self.n_player - 1)
        final_score = [self.min_score + step*rank for rank in ranking]
        return final_score
    
    def get_legal_action(self):
        if not self.is_continue:
            print('Game over, no legal action')
            return []
        current_player = self.players[self.turn]
        if current_player.chip > 0:
            return ['pass', 'take']
        else:
            return ['take']

    def next_player(self, move):
        """get next player if take then turn does not change"""
        if move == 'take':
            return self.turn
        elif move == 'pass':
            return self.turn + 1 if self.turn + 1 < self.n_player else 0
            
    def action(self, move):
        if not self.is_continue:
            print('Game over, cannot act')
            return False
        current_player = self.players[self.turn]
        if move == 'take':
            current_player.card.append(self.current_card)
            current_player.chip += self.chip_in_pot
            self.chip_in_pot = 0
            is_continue = self.flip_card()
            self.is_continue = is_continue
            return self.is_continue

        elif move == 'pass':
            current_player.chip -= 1
            self.chip_in_pot += 1
            self.turn = self.next_player(move)
            return True
    
    def rollout_policy_rule(self):
        legal_action = self.get_legal_action()
        weight = [self.rollout_rule.get(i) for i in legal_action]
        move = random.choices(legal_action, weight)[0]
        return move
    
    def rollout_policy_1(self, verbose = False):
        # current_player = self.players[self.turn]
        legal_action = self.get_legal_action()
        const = 0.5
        prob = (self.chip_in_pot/self.current_card)/const
        weight = [prob if i == 'take' else 1 - prob for i in legal_action]
        move = random.choices(legal_action, weight)[0]
        if verbose:
            print(legal_action, weight)
        return move
    
    def rollout_policy_2(self, verbose = False):
        # current_player = self.players[self.turn]
        legal_action = self.get_legal_action()
        remain = self.current_card//2 - self.chip_in_pot
        if remain <= 1:
            prob = 0.9
        else:
            prob = 0.01

        weight = [prob if i == 'take' else 1 - prob for i in legal_action]
        move = random.choices(legal_action, weight)[0]
        if verbose:
            print(legal_action, weight)
        return move
    
    def rollout_policy_3(self, p = 0.98, verbose = False):
        current_player = self.players[self.turn]
        other_card = [i for i in self.played_card if i not in current_player.card]
        good_for_me = any([abs(self.current_card - card) <= 2 for card in current_player.card])
        good_for_them = any([abs(self.current_card - card) <= 2 for card in other_card])
        least_chip = min([self.players[turn].chip for turn in range(self.n_player) if turn != self.turn])
        legal_action = self.get_legal_action()

        if good_for_me:
            if good_for_them:
                #then you must take or the other guy will take it
                good_action = 'take'
            else:
                #how much can i farm
                # look at the guy with the least chip
                good_action = 'take'
                if least_chip >= 3:
                    good_action = 'pass'
        else:
            # can i afford to pass it till taken?
            good_action = 'pass'
            if current_player.chip <= 2 or self.chip_in_pot >= self.current_card//2:
                good_action = 'take'

        weight = [p if act == good_action else 1 - p for act in legal_action]
        move = random.choices(legal_action, weight)[0]
        if verbose:
            print(legal_action, weight)
        return move


    def self_play(self, verbose = False):
        """Keeps playing till the game end
        This is where you set your rollout policy
        """
        while self.is_continue:
            # move = self.rollout_policy_rule()
            move = self.rollout_policy_3(verbose = verbose)
            if verbose:
                print(f'''Card: {self.current_card} | Chip in pot: {self.chip_in_pot} | Player: {self.turn} - {self.players[self.turn]}
move: {move}'''
    )
            self.is_continue = self.action(move)        
        score_list = self.calculate_ranking()
        return score_list

In [ ]:
class game_state:
    """This class is the node of the tree"""
    def __init__(self, parent = None, 
                #  game = None, 
                turn: int = 0,
                 depth = 0):
        self.depth = depth
        self.win = 0
        self.n_explored = 0
        self.parent = parent
        self.child = [] # (uct, move, next node)
        # self.game = game
        self.simulation_score = [0,0,0]
        self.turn = turn # need this as reference when doing backpropagation

    def calculate_uct(self):
        """Can set your uct policy here"""
        if self.n_explored == 0:
            return np.inf
        else:
            exploitation = self.win/self.n_explored
            exploration = np.sqrt(4*np.log(self.parent.n_explored)/self.n_explored)
            return exploitation + exploration
        
    def get_best_move(self):
        win_list = [child_node[-1].win/child_node[-1].n_explored for child_node in self.child]
        return self.child[np.argmax(win_list)][1]

In [ ]:
class mcts():
    def __init__(self, 
                 depth = 20, 
                 branch = 0):
        """Depth: limit on the number of card flip to look in the future"""
        self.depth = depth
        self.branch = branch
        self.simulation_iter = 100

    def get_best_uct(self, child_list):
        index_max = np.argmax([i[0] for i in child_list])
        return child_list[index_max]
    
    def simulation(self, game, iters = 100):
        score_list = np.array([0.]*game.n_player)
        for iter_no in range(iters):
            game_tmp = copy.deepcopy(game)            
            score_list_tmp = game_tmp.self_play()
            score_list += score_list_tmp
            # print(iter_no, score_list_tmp, score_list)
        return score_list


    def selection(self, game_node, game):
        # game = game_node.game
        score_list, total_explored = np.array([0.]*game.n_player), 0
        # if game_node.depth > self.depth:
        #     print('exceed depth')
        #     return score_list, total_explored

        if len(game_node.child) == 0:
            # EXPANSION
            # if game_node.depth < self.depth: # stop expanding -> switch to simulation
            legal_move = game.get_legal_action()
            for move in legal_move:     
                turn_tmp = game.next_player(move)
                next_node = game_state(parent = game_node,
                                        # game = game_clone,
                                        turn = turn_tmp,
                                    #    depth = game_node.depth + 1 if move == 'take' else game_node.depth
                                       depth = game_node.depth + 1
                                       )
                uct_tmp = next_node.calculate_uct()
                heapq.heappush(game_node.child, [uct_tmp, move, next_node])

            # SIMULATION
            total_explored = 0
            for _, move, child in game_node.child:
                game_clone = copy.deepcopy(game) # no need this
                game_clone.action(move)
                score_list_tmp = self.simulation(game_clone, self.simulation_iter)
                child.simulation_score = score_list_tmp
                child.n_explored += self.simulation_iter
                child.win += score_list_tmp[child.parent.turn]

                score_list += score_list_tmp
                total_explored += self.simulation_iter
            
            # Update the current node here too
            game_node.n_explored += total_explored
            if game_node.parent:
                game_node.win += score_list[game_node.parent.turn]
            return score_list, total_explored 

        else:
            #SELECTION
            # print(f'Game: {game} ---------------')
            for child in game_node.child:
                # print(f'Depth: {child[-1].depth} |move: {child[1]} | win: {child[-1].win } | explored: {child[-1].n_explored}| win pct: {child[-1].win/child[-1].n_explored} | uct: {child[-1].calculate_uct()}')
                child[0] = child[-1].calculate_uct()
            
            _, move, next_node = self.get_best_uct(game_node.child)
            game_clone = copy.deepcopy(game) # no need this
            game_clone.action(move)
            score_list, total_explored = self.selection(next_node, game_clone)
            
            #BACKPROPAGAION
            game_node.n_explored += total_explored
            if game_node.parent:
                game_node.win += score_list[game_node.parent.turn]

            return score_list, total_explored

In [8]:
# test self play

In [124]:
a = game()

In [ ]:
a.self_play()

In [138]:
# test mcts

In [ ]:
nothanks = game(32)
game_node = game_state()
tree = mcts()

In [ ]:
print(nothanks)
while nothanks.is_continue:
    for _ in tqdm(range(50)):
        tree.selection(game_node= game_node, game= nothanks)
    move = game_node.get_best_move()
    print(f'''Card: {nothanks.current_card} | Chip in pot: {nothanks.chip_in_pot} | Player: {nothanks.turn} - {nothanks.players[nothanks.turn]}
move: {move}'''
)
    nothanks.action(move)
    game_node = game_state(turn = nothanks.turn)

In [16]:
nothanks.calculate_ranking()

[1.0, 0.0, -1.0]

In [20]:
for user in nothanks.players:
    print(user, user.calculate_score())

Chip: 0 | Card owned: [4, 13, 22, 26] -65
Chip: 29 | Card owned: [10, 20, 30, 32, 29, 11, 5, 6, 28, 16, 9, 31, 19, 15, 25, 12] -72
Chip: 4 | Card owned: [3, 35, 27, 17] -78
